# Lecture 16: Writing Functions in R
<div style="border: 1px double black; padding: 10px; margin: 10px">

**After today's lecture you will understand:**
* How to write functions in R
</div>

These notes correspond to Chapter 26 of the book.





In [96]:
library(tidyverse)
remotes::install_github("bradleyboehmke/harrypotter")
install.packages("tidytext")
library(harrypotter)
library(tidytext)
load(url('https://datasets.stats306.org/afinn.RData'))

Skipping install of 'harrypotter' from a github remote, the SHA1 (51f71461) has not changed since last install.
  Use `force = TRUE` to force installation

Installing package into ‘/home/terhorst/R/x86_64-pc-linux-gnu-library/4.2’
(as ‘lib’ is unspecified)



## Why get better at programming?

This week we will learn how to become better R programmers, by studying functions and iteration. The goal is to learn how to write code that is simulataneously:
- Easier to read;
- Less buggy;
- More enjoyable to write;
- More efficient.

## Example from HW7

<pre style="font-size: 8px; line-height: 1;">
chamber_tibble<-tibble(chapter=seq_along(chamber_of_secrets), text=chamber_of_secrets)%>%
  unnest_tokens(input=text, output=word)%>%
  left_join(afinn)%>%
  filter(!is.na(value))%>%
  group_by(chapter)%>%
  summarize(mean_sentiment_value=mean(value,na.rm=T))%>%
  mutate(book="2.chamber_of_secrets")

prisoner_tibble<-tibble(chapter=seq_along(prisoner_of_azkaban), text=prisoner_of_azkaban)%>%
  unnest_tokens(input=text, output=word)%>%
  left_join(afinn)%>%
  filter(!is.na(value))%>%
  group_by(chapter)%>%
  summarize(mean_sentiment_value=mean(value,na.rm=T))%>%
  mutate(book="3.prisoner_of_azkaban")

goblet_tibble<-tibble(chapter=seq_along(goblet_of_fire), text=goblet_of_fire)%>%
  unnest_tokens(input=text, output=word)%>%
  left_join(afinn)%>%
  filter(!is.na(value))%>%
  group_by(chapter)%>%
  summarize(mean_sentiment_value=mean(value,na.rm=T))%>%
  mutate(book="4.goblet_of_fire")

phoenix_tibble<-tibble(chapter=seq_along(order_of_the_phoenix), text=order_of_the_phoenix)%>%
  unnest_tokens(input=text, output=word)%>%
  left_join(afinn)%>%
  filter(!is.na(value))%>%
  group_by(chapter)%>%
  summarize(mean_sentiment_value=mean(value,na.rm=T))%>%
  mutate(book="5.order_of_the_phoenix")

prince_tibble<-tibble(chapter=seq_along(half_blood_prince), text=half_blood_prince)%>%
  unnest_tokens(input=text, output=word)%>%
  left_join(afinn)%>%
  filter(!is.na(value))%>%
  group_by(chapter)%>%
  summarize(mean_sentiment_value=mean(value,na.rm=T))%>%
  mutate(book="6.half_blood_prince")

hallows_tibble<-tibble(chapter=seq_along(deathly_hallows), text=deathly_hallows)%>%
  unnest_tokens(input=text, output=word)%>%
  left_join(afinn)%>%
  filter(!is.na(value))%>%
  group_by(chapter)%>%
  summarize(mean_sentiment_value=mean(value,na.rm=T))%>%
  mutate(book="7.deathly_hallows")
</pre>

In [97]:
ls("package:harrypotter") %>% 
    map_dfr(~ tibble(title = ., text = getExportedValue('harrypotter', .), 
                     chapter = seq_along(text))) %>% 
    unnest_tokens(input=text, output=word) %>% 
    left_join(afinn) %>%
    group_by(title, chapter) %>% 
    summarize(msv = mean(value, na.rm=T), .groups = "drop") %>% 
    top_n(5, -msv)

Joining with `by = join_by(word)`


title,chapter,msv
<chr>,<int>,<dbl>
goblet_of_fire,32,-1.0439560
goblet_of_fire,34,-0.7664671
half_blood_prince,28,-1.1428571
order_of_the_phoenix,36,-0.9166667
prisoner_of_azkaban,17,-0.7350000


# Function
This lecture will be about writing our own functions. 
- Functions are not new to us; we have been using them since the very first lecture.
- `print`, `filter`, `mutate`, `summarize`, `tibble`, etc.: all are functions.

In [101]:
class(filter)
class(1:3)

[1] "function"

[1] "integer"

### Why write our own functions??

Often when programming we find ourselves repeating the same block of code with minor modifications. 

Did you encounter this situation in your HW7 when you applied the same logic again and again, while deriving the sentiment score for all the Harry Potter books??

That is precisely when a function is defined to save ourselves from repetition!

Let us start with simple examples. When building machine learning models (which you will learn next week) it is common practice to normalize all the columns values to the same scale; typically between 0 and 1. Let us take the `mpg` dataset and see its current min and max values:

In [104]:
range(mpg$hwy)
c(min(mpg$hwy), max(mpg$hwy))

[1] 12 44

[1] 12 44

Let us now normalize it:

In [106]:
hwy_a <- (mpg$hwy - min(mpg$hwy, na.rm = TRUE)) / (max(mpg$hwy, na.rm = TRUE) - min(mpg$hwy, na.rm = TRUE))
hwy_a

[1] 0.53125 0.53125 0.59375 0.56250 0.43750 0.43750 0.46875 0.43750 0.40625
 [10] 0.50000 0.46875 0.40625 0.40625 0.40625 0.40625 0.37500 0.40625 0.34375
 [19] 0.25000 0.09375 0.25000 0.15625 0.15625 0.43750 0.34375 0.43750 0.40625
 [28] 0.37500 0.21875 0.06250 0.09375 0.15625 0.46875 0.56250 0.43750 0.53125
 [37] 0.43750 0.37500 0.37500 0.31250 0.31250 0.37500 0.37500 0.15625 0.31250
 [46] 0.28125 0.34375 0.34375 0.21875 0.18750 0.15625 0.15625 0.21875 0.21875
 [55] 0.00000 0.15625 0.09375 0.15625 0.15625 0.00000 0.15625 0.12500 0.18750
 [64] 0.09375 0.12500 0.00000 0.15625 0.15625 0.12500 0.00000 0.09375 0.12500
 [73] 0.15625 0.09375 0.15625 0.15625 0.18750 0.15625 0.21875 0.15625 0.21875
 [82] 0.21875 0.15625 0.15625 0.15625 0.12500 0.12500 0.15625 0.09375 0.15625
 [91] 0.43750 0.40625 0.43750 0.37500 0.28125 0.31250 0.34375 0.31250 0.25000
[100] 0.65625 0.62500 0.62500 0.53125 0.62500 0.68750 0.75000 0.75000 0.53125
[109] 0.43750 0.46875 0.56250 0.59375 0.43750 0.43750 0.50000 0.43750 0.53125
[118] 0.50000 0.46875 0.37500 0.37500 0.37500 0.31250 0.21875 0.25000 0.15625
[127] 0.00000 0.21875 0.18750 0.06250 0.09375 0.18750 0.18750 0.09375 0.15625
[136] 0.12500 0.18750 0.15625 0.21875 0.21875 0.15625 0.53125 0.46875 0.59375
[145] 0.62500 0.46875 0.43750 0.43750 0.40625 0.40625 0.15625 0.15625 0.25000
[154] 0.18750 0.43750 0.43750 0.46875 0.50000 0.40625 0.40625 0.37500 0.46875
[163] 0.40625 0.43750 0.34375 0.43750 0.43750 0.43750 0.43750 0.40625 0.46875
[172] 0.40625 0.46875 0.25000 0.25000 0.21875 0.15625 0.25000 0.15625 0.53125
[181] 0.46875 0.59375 0.59375 0.43750 0.43750 0.50000 0.46875 0.53125 0.59375
[190] 0.59375 0.43750 0.43750 0.46875 0.56250 0.65625 0.71875 0.78125 0.71875
[199] 0.09375 0.18750 0.25000 0.25000 0.31250 0.15625 0.21875 0.18750 0.25000
[208] 0.53125 0.43750 0.53125 0.53125 0.37500 1.00000 0.53125 0.43750 0.53125
[217] 0.53125 0.53125 0.53125 0.34375 0.37500 1.00000 0.90625 0.53125 0.43750
[226] 0.50000 0.53125 0.53125 0.53125 0.50000 0.53125 0.43750 0.43750 0.43750

We need to do the exact same for all columns; to begin with let us do for all numerical columns. And it is easy to copy and paste the same code from above and change the variable names

## 🤔 Quiz

In [115]:
cty_a <- (
    (mpg$cty - min(mpg$cty, na.rm = TRUE)) / 
    (max(mpg$cty, na.rm = TRUE) - min(mpg$hwy, na.rm = TRUE))
)
cty_a %>% range

[1] 0.000000 1.130435

Why is the range not showing a maximum of 1?

&nbsp;A. For the `cty` values it cannot have a maximum value of 1<br/>
&nbsp;B. The numerator should use `max` function<br/>
&nbsp;C. Copy-paste led to making an accidental mistake in the formula.<br/>

We still need to do the exact same procedure for all other columns as well and so intead of repeating ourselves we can introduce a function to do the repeats. 

Specifically, we want to create a procedure that "fills in" the black boxes in the pattern below with whatever we pass in:

(█ - min(█, na.rm = TRUE)) / (max(█, na.rm = TRUE) - min(█, na.rm = TRUE))

## Anatomy of a function

To write a function we should first think about the inputs and output. A function takes input(s), does something(s) to them, and then returns an output.


    f <- function(input) {
        <do some things>
        return(output)
    }
    output <- f(input)



Now let's create a "rescale" function from the above code:

In [138]:
# rescale01 function

rescale01 <- function(x, na.rm = FALSE) {
    # subtract off the min, divide by the range
    return((x - min(x, na.rm = na.rm)) / (max(x, na.rm = na.rm) - min(x, na.rm = na.rm)))
}

rescale01(c(NA, mpg$cty), na.rm = T)

[1]         NA 0.34615385 0.46153846 0.42307692 0.46153846 0.26923077
  [7] 0.34615385 0.34615385 0.34615385 0.26923077 0.42307692 0.38461538
 [13] 0.23076923 0.30769231 0.30769231 0.23076923 0.23076923 0.30769231
 [19] 0.26923077 0.19230769 0.07692308 0.19230769 0.15384615 0.11538462
 [25] 0.26923077 0.23076923 0.26923077 0.23076923 0.23076923 0.19230769
 [31] 0.07692308 0.07692308 0.19230769 0.38461538 0.50000000 0.34615385
 [37] 0.34615385 0.30769231 0.34615385 0.30769231 0.26923077 0.26923077
 [43] 0.30769231 0.30769231 0.07692308 0.23076923 0.23076923 0.26923077
 [49] 0.26923077 0.23076923 0.19230769 0.15384615 0.19230769 0.19230769
 [55] 0.19230769 0.00000000 0.07692308 0.07692308 0.15384615 0.15384615
 [61] 0.00000000 0.15384615 0.07692308 0.15384615 0.07692308 0.11538462
 [67] 0.00000000 0.15384615 0.15384615 0.11538462 0.00000000 0.07692308
 [73] 0.07692308 0.15384615 0.07692308 0.07692308 0.07692308 0.11538462
 [79] 0.19230769 0.23076923 0.19230769 0.15384615 0.15384615 0.15384615
 [85] 0.19230769 0.19230769 0.15384615 0.15384615 0.15384615 0.07692308
 [91] 0.15384615 0.34615385 0.34615385 0.30769231 0.26923077 0.23076923
 [97] 0.23076923 0.23076923 0.23076923 0.19230769 0.73076923 0.57692308
[103] 0.61538462 0.53846154 0.57692308 0.65384615 0.61538462 0.57692308
[109] 0.46153846 0.34615385 0.34615385 0.46153846 0.46153846 0.34615385
[115] 0.34615385 0.38461538 0.38461538 0.38461538 0.42307692 0.42307692
[121] 0.30769231 0.26923077 0.30769231 0.30769231 0.23076923 0.23076923
[127] 0.19230769 0.00000000 0.19230769 0.15384615 0.07692308 0.07692308
[133] 0.11538462 0.11538462 0.07692308 0.07692308 0.07692308 0.11538462
[139] 0.19230769 0.15384615 0.15384615 0.15384615 0.46153846 0.38461538
[145] 0.53846154 0.53846154 0.38461538 0.38461538 0.34615385 0.38461538
[151] 0.38461538 0.19230769 0.23076923 0.19230769 0.11538462 0.34615385
[157] 0.26923077 0.30769231 0.34615385 0.26923077 0.34615385 0.34615385
[163] 0.42307692 0.38461538 0.42307692 0.34615385 0.46153846 0.38461538
[169] 0.38461538 0.38461538 0.42307692 0.42307692 0.38461538 0.42307692
[175] 0.23076923 0.26923077 0.23076923 0.23076923 0.26923077 0.19230769
[181] 0.46153846 0.46153846 0.46153846 0.46153846 0.34615385 0.34615385
[187] 0.38461538 0.46153846 0.46153846 0.46153846 0.50000000 0.34615385
[193] 0.34615385 0.34615385 0.57692308 0.57692308 0.65384615 0.73076923
[199] 0.65384615 0.07692308 0.15384615 0.23076923 0.26923077 0.30769231
[205] 0.23076923 0.23076923 0.23076923 0.26923077 0.46153846 0.38461538
[211] 0.46153846 0.50000000 0.30769231 0.92307692 0.46153846 0.38461538
[217] 0.50000000 0.46153846 0.46153846 0.46153846 0.26923077 0.30769231
[223] 1.00000000 0.76923077 0.46153846 0.38461538 0.42307692 0.42307692
[229] 0.46153846 0.34615385 0.38461538 0.46153846 0.26923077 0.34615385
[235] 0.30769231

Note: in R (and unlike Python, C++, Java, or other languages), it is not necessary to use a `return` keyword to specify the return value of a function. The return value of a function is simply the value of the last expression evaluated within the function body. 

## 🤔 Quiz

What is the output of our `rescale01` function?

* A. x
* B. nothing
* C. rescaled vector values



### Arguments

Functions will often have multiple arguments. Some arguments have default values, others do not. All arguments without default values must be passed to a function. Arguments can be passed by name or position. For instance, 




In [126]:
# generate 5 numbers from a Normal(0, 1) distribution.
w = rnorm(5, mean = 0, sd = 1)

Arguments passed by name need not be in order:

In [34]:
w = rnorm(mean = 0, sd = 1, n = 5)
u = rnorm(mean = 0, sd = 1, 5) # This also works but is bad style. 
round(rbind(u = u, w = w), 1)

# unnamed arguments get passed to the first argument after the names arguments are assigned

u,0.2,-0.1,1.8,-1.2,-0.1
w,1.7,-1.2,0.5,-1.2,-0.1


## Vector functions
The first type of function we'll look at are functions that take a vector, and maybe some other options, and return a vector. We follow the book and call these "vector functions".

### $z$-score function

Let us create a function to compute $z$-scores of a vector:

In [208]:
help(mean)

In [213]:
# function to compute z-scores
z_score = function(x, na.rm = F) {
    # (x - mean) / sd
    return(
        (x - mean(x, na.rm = na.rm)) / sd(x, na.rm = na.rm)
    )
}

z_score(c(NA, mpg$cty))

[1] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
 [26] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
 [51] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
 [76] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
[101] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
[126] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
[151] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
[176] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
[201] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
[226] NA NA NA NA NA NA NA NA NA NA

The return statement is not strictly necessary, but can make complex functions more readable. It is good practice to avoid creating intermediate objects to store values only used once.



#### Parameters

We can set default values for parameters using the construction `parameter = xx` in the function definition.




In [9]:
# function to compute z-scores, while potentially remove NAs
z_score = function(x, na.rm){
    # complete
}

Sometimes it makes sense to set a default value for the parameter. We can do this by specifying the default afterwards:

In [12]:
# function to compute z-scores, while potentially remove NAs
z_score = function(x, na.rm = F){
    # complete
}

Is it better to make the default `na.rm=True` or `na.rm=False`?

### Coefficient of variation
The coefficient of variation is defined as the ratio of a (numerical) variable's standard deviation to its mean.

In [151]:
cv <- function(x) {
    return(sd(x) / mean(x))
}

cv(mpg$cty)

[1] 0.2524439

### How old
Write a function that, given a vector of people's birthdays, returns a vector of how old each person is (in years):

In [159]:
library(lubridate)

how_old <- function(birthdays) {
    today() - birthdays
}

how_old(mdy(c("09/26/1982","09/22/2019")))

Time differences in days
[1] 14785  1275

## Summarize functions
The preceding functions all took a vector as input, and returned a vector of the same length as output. These are appropriate for use in operations like **mutate** and **filter**, which expect a function to behave in such a way. 

A second type of function takes a vector and returns a single number. These are suitable for use with the **summarize** function.

###  Example: skewness
The _skewness_ of a random variable is defined as $$\mathbb{E} \left( \frac{X - \mu}{\sigma} \right)^3.$$

Let's write a function that takes a vector of numerical values and computes its skewness:

In [165]:
skewness <- function(x) {
    mean((x - mean(x) / sd(x))^3)
}

mpg %>% group_by(class) %>% summarize(s_cty = rescale01(cty), s_hwy = rescale01(hwy))

Warning message:
“Returning more (or less) than 1 row per `summarise()` group was deprecated in dplyr 1.1.0.
ℹ Please use `reframe()` instead.
ℹ When switching from `summarise()` to `reframe()`, remember that `reframe()` always returns an ungrouped data frame and adjust accordingly.”
`summarise()` has grouped output by 'class'. You can override using the `.groups` argument.


class,s_cty,s_hwy
<chr>,<dbl>,<dbl>
2seater,1.00000000,1.0000000
2seater,0.00000000,0.0000000
2seater,1.00000000,1.0000000
2seater,0.00000000,0.6666667
2seater,0.00000000,0.3333333
compact,0.16666667,0.2857143
compact,0.33333333,0.2857143
compact,0.27777778,0.3809524
compact,0.33333333,0.3333333


Now let's use this function in a summary.

In [50]:
# skewness inside of summary

## Dataframe functions

The second class of functions we'll look at take a _dataframe_ as input, and also return a _dataframe_. We have already seen many examples of this type of function:
- mutate
- filter
- summarize
- arrange
- select


A case of a problem of `indirection`, as `dplyr` uses `tidy evaluation` to allow you to refer to the names of variables inside your data frame without any special treatment.

Let's consider writing a function called "grouped_mean" that takes a data frame, a grouping variable, and a mean variable, and summarizes it. We might think to write it like this:

In [169]:
grouped_mean <- function(df, group_var, mean_var) {
  df %>%
    group_by(group_var) %>% 
    summarize(mean(mean_var))
}

mpg %>% grouped_mean(class, hwy)


ERROR: [1m[33mError[39m in `group_by()`:[22m
[1m[22m[33m![39m Must group by variables found in `.data`.
[31m✖[39m Column `group_var` is not found.


However, this won't quite work:

In [170]:
grouped_mean(mpg, model, hwy)

ERROR: [1m[33mError[39m in `group_by()`:[22m
[1m[22m[33m![39m Must group by variables found in `.data`.
[31m✖[39m Column `group_var` is not found.


### Embracing `{{ }}`
The reason why this doesn't work is that R does not understand that `model` and `hwy` are columns inside of `mpg`. (This is the way most computer languages work.) To fix this, R has a very special syntax for indicating that certain arguments to the function are columns inside of a data frame:


In [173]:
grouped_mean <- function(df, group_var, mean_var) {
  df %>%
    group_by({{ group_var }}) %>% 
    summarize(mean({{ mean_var }}))
}
mpg %>% grouped_mean(model, hwy) %>% head

model,mean(hwy)
<chr>,<dbl>
4runner 4wd,18.83333
a4,28.28571
a4 quattro,25.75000
a6 quattro,24.00000
altima,28.66667
c1500 suburban 2wd,17.80000


This is called _embracing_ in the book.

Now let's try a simple modification of our `grouped_mean` function, that allows use to group over _multiple_ variables:

In [178]:
grouped_mean <- function(df, group_vars, mean_var) {
  df %>%
    group_by(pick({{ group_vars }})) %>% 
    summarize(mean({{ mean_var }}))
}
mpg %>% grouped_mean(model:class, hwy) %>% head

`summarise()` has grouped output by 'model', 'displ', 'year', 'cyl', 'trans', 'drv', 'cty', 'hwy', 'fl'. You can override using the `.groups` argument.


model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mean(hwy)
<chr>,<dbl>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>
4runner 4wd,2.7,1999,4,auto(l4),4,16,20,r,suv,20
4runner 4wd,2.7,1999,4,manual(m5),4,15,20,r,suv,20
4runner 4wd,3.4,1999,6,auto(l4),4,15,19,r,suv,19
4runner 4wd,3.4,1999,6,manual(m5),4,15,17,r,suv,17
4runner 4wd,4.0,2008,6,auto(l5),4,16,20,r,suv,20
4runner 4wd,4.7,2008,8,auto(l5),4,14,17,r,suv,17


Why did we get this error? It turns out there are two forms of "embracing":

* Data-masking: this is used in functions like arrange(), filter(), and summarize() that compute with variables.

* Tidy-selection: this is used for functions like select(), relocate(), and rename() that select variables.

You can tell which functions take which by looking at the help file. But generally, your intuition is correct: functions that do computations on columns of data are "data-masking"; functions that select columns are "tidy-selection".

To fix our `grouped_mean` function, we'll use the `pick()` function, which lets you use tidy-selection inside data-masking functions:

In [59]:
grouped_mean <- function(df, group_vars, mean_var) {
    # grouped_mean using pick
}

grouped_mean(mpg, c(model, class), hwy) %>% head

`summarise()` has grouped output by 'model'. You can override using the `.groups` argument.


model,class,mean(hwy)
<chr>,<chr>,<dbl>
4runner 4wd,suv,18.83333
a4,compact,28.28571
a4 quattro,compact,25.75000
a6 quattro,midsize,24.00000
altima,compact,28.00000
altima,midsize,29.00000


## Abstraction

The main reason to use functions is abstraction: they enable us to break down complex problems into smaller pieces, that we can then reason about individually. 

![example of a stack trace](http://2.bp.blogspot.com/-9nBb0CvqBIg/T2UKV06nD5I/AAAAAAAAAkQ/Pl2Hfj5HUlY/s1600/short-stack.png)

Let's try this principle out on a problem from last ~night's~week's homework:

    Consider the Harry Potter series as a collection of    
    documents(books), find top 5 words in each book ranked
    by TF-IDF. Comment on your finding. (1 point)

A possible solution:
<pre style="font-size: 6px; line-height: 1">
phil_words<-tibble(text=philosophers_stone)%>%
  unnest_tokens(input=text, output=word)%>%mutate(book=1)
chamber_words<-tibble(text=chamber_of_secrets)%>%
  unnest_tokens(input=text, output=word)%>%mutate(book=2)
prisoner_words<-tibble(text=prisoner_of_azkaban)%>%
  unnest_tokens(input=text, output=word)%>%mutate(book=3)
goblet_words<-tibble(text=goblet_of_fire)%>%
  unnest_tokens(input=text, output=word)%>%mutate(book=4)
phoenix_words<-tibble(text=order_of_the_phoenix)%>%
  unnest_tokens(input=text, output=word)%>%mutate(book=5)
prince_words<-tibble(text=half_blood_prince)%>%
  unnest_tokens(input=text, output=word)%>%mutate(book=6)
hallows_words<-tibble(text=deathly_hallows)%>%
  unnest_tokens(input=text, output=word)%>%mutate(book=7)

wordlist<-phil_words%>%bind_rows(chamber_words,prisoner_words,goblet_words,phoenix_words,prince_words,hallows_words)%>%
  group_by(word)%>%
  summarize(k=n_distinct(book))

phil_tbl<-phil_words%>%left_join(wordlist)%>%
  group_by(word,k)%>%
  count()%>%
  mutate(IDF=1+log(7/k),TF=log(1+n))%>%
  mutate(TF_IDF=TF*IDF)%>%
  ungroup%>%
  arrange(desc(TF_IDF))

chamber_tbl<-chamber_words%>%left_join(wordlist)%>%
  group_by(word,k)%>%
  count()%>%
  mutate(IDF=1+log(7/k),TF=log(1+n))%>%
  mutate(TF_IDF=TF*IDF)%>%
  ungroup%>%
  arrange(desc(TF_IDF))

prisoner_tbl<-prisoner_words%>%left_join(wordlist)%>%
  group_by(word,k)%>%
  count()%>%
  mutate(IDF=1+log(7/k),TF=log(1+n))%>%
  mutate(TF_IDF=TF*IDF)%>%
  ungroup%>%
  arrange(desc(TF_IDF))

goblet_tbl<-goblet_words%>%left_join(wordlist)%>%
  group_by(word,k)%>%
  count()%>%
  mutate(IDF=1+log(7/k),TF=log(1+n))%>%
  mutate(TF_IDF=TF*IDF)%>%
  ungroup%>%
  arrange(desc(TF_IDF))

phoenix_tbl<-phoenix_words%>%left_join(wordlist)%>%
  group_by(word,k)%>%
  count()%>%
  mutate(IDF=1+log(7/k),TF=log(1+n))%>%
  mutate(TF_IDF=TF*IDF)%>%
  ungroup%>%
  arrange(desc(TF_IDF))

prince_tbl<-prince_words%>%left_join(wordlist)%>%
  group_by(word,k)%>%
  count()%>%
  mutate(IDF=1+log(7/k),TF=log(1+n))%>%
  mutate(TF_IDF=TF*IDF)%>%
  ungroup%>%
  arrange(desc(TF_IDF))

hallows_tbl<-hallows_words%>%left_join(wordlist)%>%
  group_by(word,k)%>%
  count()%>%
  mutate(IDF=1+log(7/k),TF=log(1+n))%>%
  mutate(TF_IDF=TF*IDF)%>%
  ungroup%>%
  arrange(desc(TF_IDF))

phil_tbl%>%top_n(5,TF_IDF)
chamber_tbl%>%top_n(5,TF_IDF)
prisoner_tbl%>%top_n(5,TF_IDF)
goblet_tbl%>%top_n(5,TF_IDF)
phoenix_tbl%>%top_n(5,TF_IDF)
prince_tbl%>%top_n(5,TF_IDF)
hallows_tbl%>%top_n(5,TF_IDF)
</pre>

Let's think abstractly about how to solve this problem:

In [182]:
hp_tbl <- ls("package:harrypotter") %>% 
    map_dfr(~ tibble(book = ., text = getExportedValue('harrypotter', .), 
                     chapter = seq_along(text)))

hp_tbl %>% tail %>% print

# A tibble: 6 × 3
  book                text                                               chapter
  <chr>               <chr>                                                <int>
1 prisoner_of_azkaban "　　CAT, RAT, AND DOG　　Harry's mind had gone b…      17
2 prisoner_of_azkaban "　　MOONY, WORMTAIL, PADDFOOT, AND PRONGS　　It …      18
3 prisoner_of_azkaban "　　THE SERVANT OF LORD VOLDEMORT　　Hermione sc…      19
4 prisoner_of_azkaban "　　THE DEMENTOR'S KISS　　Harry had never been …      20
5 prisoner_of_azkaban "　　HERMIONE'S SECRET　　Shocking business... sh…      21
6 prisoner_of_azkaban "　　OWL POST AGAIN　　Harry!\"　　Hermione was t…      22


In [196]:
# TF-IDF(t,d) = TF(t,d)×IDF(t)
# TF(t,d) = log(1 + c(t,d))
# IDF(t) = log(N/k)

convert_to_words <- function(df) {
    unnest_tokens(df, input=text, output=word)
}

number_of_books <- function(df) {
    # return a new df with k=# of books each word appeared in
    df %>% convert_to_words %>% group_by(word) %>% summarize(k = n_distinct(book))
}

tf <- hp_tbl %>% convert_to_words %>% count(book, word) %>% mutate(tf = log(1 + n))
idf <- hp_tbl %>% number_of_books %>% mutate(idf = log(7 / k))

tf_idf <- inner_join(tf, idf) %>% mutate(tf_idf = tf * idf)

Joining with `by = join_by(word)`


In [197]:
tf_idf

book,word,n,tf,k,idf,tf_idf
<chr>,<chr>,<int>,<dbl>,<int>,<dbl>,<dbl>
chamber_of_secrets,0,1,0.6931472,3,0.8472979,0.5873021
chamber_of_secrets,1,6,1.9459101,6,0.1541507,0.2999634
chamber_of_secrets,"1,5",1,0.6931472,1,1.9459101,1.3488021
chamber_of_secrets,"1,520",1,0.6931472,1,1.9459101,1.3488021
chamber_of_secrets,100,1,0.6931472,1,1.9459101,1.3488021
chamber_of_secrets,101,1,0.6931472,1,1.9459101,1.3488021
chamber_of_secrets,102,1,0.6931472,1,1.9459101,1.3488021
chamber_of_secrets,104,1,0.6931472,1,1.9459101,1.3488021
chamber_of_secrets,105,1,0.6931472,1,1.9459101,1.3488021


In [95]:
# TF-IDF(t,d) = TF(t,d)×IDF(t)
# TF(t,d) = log(1 + c(t,d))
# IDF(t) = log(N/k)

tf_idf <- function(df, document_col, word_col) {
    # fill in
}

## Scope

Scoping refers to how R looks up the value associated with an object referred to by name. There are two types of scoping – lexical and dynamic – but we will concern ourselves only with lexical scoping here. There are four keys to understanding scoping:

- environments
- name masking
- variables vs functions
- dynamic look up 


An environment can be thought of as a context in which names are associated with objects. Each time a function is called, it generates a new environment for the computation.

Consider the following examples:

In [42]:
ls()

[1] "a"          "b"          "c"          "cty"        "cty_a"     
 [6] "hwy_a"      "rescale"    "sum"        "u"          "w"         
[11] "x"          "x_centered" "y"          "z"          "z_score1"  
[16] "z_score2"

In [43]:
f1 = function() {
  f1_message = "I'm defined inside of f!"  # `message` is a function in base
  ls()
}
f1()

[1] "f1_message"

In [44]:
exists('f1') # f1 %in% ls() 

[1] TRUE

In [45]:
exists('f1_message')

[1] FALSE

In [46]:
environment() # here we are in the global environment

<environment: R_GlobalEnv>

In [47]:
f2 = function(){
  environment() # here we are in the local environment -- each time we get a different local environment
    # created for the purpose of this function
}
f2()

<environment: 0x564d05c250a0>

In [ ]:
rm(f1, f2)

Name masking refers to where and in what order `R` looks for object names.
When we call `f1` above, `R` first looks in the current environment which happens to be the global environment. The call to `ls()` however, happens within the environment created by the function call and hence returns only the objects defined in the local environment.

When an environment is created, it gets nested within the current environment referred to as the “parent environment”. When an object is referenced we first look in the current environment and move recursively up through parent environments until we find a value bound to that name.



Name masking refers to the notion that objects of the same name can exist in different environments. Consider these examples:



In [57]:
#  Example 3 -- lexical scoping
y = x = 'I came from outside of f!'
f3 = function(){
  x =  'I came from inside of f!'
  print(paste("x =", x, "and y =", y))
}
f3()
print(paste("outside-x =", x, "and outside-y =", y))

[1] "x = I came from inside of f! and y = I came from outside of f!"
[1] "outside-x = I came from outside of f! and outside-y = I came from outside of f!"


* x is redefined inside the function enviornment
* y is not, so R will search for y in the parent environment and keep moving up
* x that is associated with f3, is not going to change the x in the global environment, unless we explicitly write the code to do that

In [ ]:
#  Example 4 -- masking
mean = function(x){ 
    sum(x)
}
mean(1:10)

[1] 55

In [ ]:
base::mean(1:10)



[1] 5.5

In [ ]:
rm(mean)

R also uses dynamic look up, meaning values are searched for when a function is called, not when it is created. In the example above, y was defined in the global environment rather than within the function body. This means the value returned by f3 depends on the value of y in the global environment. You should generally avoid this, but there are occasions where it can be useful.



In [ ]:
# Example 5 - dynamic lookup
y = "I have been reinvented!"
f3()

$x
[1] "I came from inside of f!"

$y
[1] "I have been reinvented!"